In [1]:
import pandas as pd 
import numpy as np 
import pathlib as path
import matplotlib.pyplot as plt
import random
from PIL import Image
from random import sample
from cmath import nan

In [2]:

batch_name='batch 3' #batch name - make sure the folder is structured as followes: (the test results/data doesnt have to be there)

#parent name: 
# -pilot_24_hours: 
#   -batch name (folder)
#       -data (folder)
#       -Batch_encoding_batch_results.csv 
#       -Batch_testing_batch_results.csv 
#       -Batch_workers.csv


qualification_name_for_testin='UPDATE-eligible for next step memory rep'  #note that the -space- after the title is improtant as for some reason this is how the qualification name is defined
qualification_name_for_entire_experiment='UPDATE-completed memory rep'

#all data location is relational to the location of this jupiter notebook:
batch_data_location=path.Path.cwd().parent
batch_data_location = batch_data_location / 'data' / 'pilot_24_hours' 


if (batch_data_location / batch_name).is_dir():
    batch_data_location= batch_data_location / batch_name
    print('Batch found',batch_data_location)
else: 
    print(f'PROBLEM FOUND: the requested batch name {batch_name} does not apear in the folder: {batch_data_location}')


PATH_TO_DATA = batch_data_location / 'data'
if not PATH_TO_DATA.exists():
    print('There is no data folder in the requested location',batch_data_location / 'data')
else: 
    print('Data folder found',batch_data_location / 'data')




Batch found c:\Users\d_abe\Desktop\human similarity database\edens github repo\Visual-memory-task\data\pilot_24_hours\batch 3
Data folder found c:\Users\d_abe\Desktop\human similarity database\edens github repo\Visual-memory-task\data\pilot_24_hours\batch 3\data


In [7]:
shelf_dict={
  "A15B4KZ3S04HS8": [
    31,
    1667149704730,
    1667223830820,
    1667239242566
  ],
  "A1LA6CIGBNDOH9": [
    59,
    1667156876722,
    1667237966478
  ],
  "A22HIX1M4QXZBB": [
    19,
    1666804285762,
    1666882028114
  ],
  "A248QG4DPULP46": [
    2,
    1666796745860,
    1666875548743
  ],
  "A2J1DNVMJ56JG1": [
    22,
    1666806879557,
    1666887012627
  ],
  "A2J57IBR2XIWLS": [
    61,
    1667171400527,
    1667244911656,
    1667248541474
  ],
  "A3MIDLO5S7FU06": [
    7,
    1667148924846,
    1667227496622
  ],
  "A3U0GQGAPN2DAV": [
    25,
    1667148683750,
    1667239544108
  ],
  "A5P12YJP805RG": [
    1,
    1667147891427,
    1667216156787,
    1667226130459
  ],
  "ASNNAP90D5R1Z": [
    47,
    1667152406322,
    1667216854306,
    1667237576573
  ],
  "A1JJYY622DGE5L": [
    0,
    1667410475964
  ],
  "A2KDZLWD8RAHDM": [
    3,
    1667410512595
  ],
  "AE33JO53WTHZQ": [
    4,
    1667410525427
  ],
  "A4ANRSA55IW5Q": [
    5,
    1667410579171
  ],
  "A11EXIB1MVBZFJ": [
    6,
    1667410584962
  ],
  "AMEBLCWTZKLS2": [
    8,
    1667410598167
  ],
  "A3EBIC3PKUSKCL": [
    9,
    1667410616477
  ],
  "A1PN0EFQD8OHSU": [
    999,
    999,
    999
  ],
  "ASVSPBUO4YRBT": [
    11,
    1667410627238,
    1667410745681,
    1667410971272,
    1667414162527
  ],
  "A10AKR84P1WXHL": [
    999,
    999,
    999
  ],
  "AVZRZOK0F26P6": [
    12,
    1667410735707
  ],
  "A3EWBTHYFI4I6Q": [
    10,
    1667410796338,
    1667411217538,
    1667412334055
  ],
  "A1MT4FMXL4WRAF": [
    999,
    999,
    999
  ],
  "ABGKJYEITBKIL": [
    999,
    999,
    999
  ],
  "AK4WAT44YKU7J": [
    14,
    1667410895492
  ],
  "A23OK9EV00HCR4": [
    13,
    1667410896026,
    1667411400167,
    1667411936244
  ],
  "A3VHDQR8A9JJ4F": [
    15,
    1667410936684
  ],
  "AL113DDABUJQ0": [
    999,
    999,
    999
  ],
  "A1CB9NR3SN4VMY": [
    999,
    999,
    999
  ],
  "AB8XECKH1JO8P": [
    18,
    1667411003183
  ],
  "A2HY7GQ07YIZTT": [
    16,
    1667411105519
  ],
  "A33PJ8605347GY": [
    17,
    1667411126127,
    1667411219518
  ],
  "A1W0K28IND6NR4": [
    20,
    1667411170189
  ],
  "A2U50SMRZT60ZF": [
    21,
    1667411316900
  ],
  "A1U0FDPQ953KXX": [
    23,
    1667411372469
  ],
  "AZNWNTQ47L88M": [
    24,
    1667411391333,
    1667412316025,
    1667412683374,
    1667412895440
  ],
  "A2M183CETUMR96": [
    26,
    1667411416274
  ],
  "A1LCUPRZ0I8S3I": [
    27,
    1667411428866,
    1667411954191
  ],
  "A1L5A88C9PPK5L": [
    28,
    1667411482584,
    1667411924868
  ],
  "A3UZ5V4Y0K3YPV": [
    29,
    1667411545745
  ],
  "A2FQLA1NF502GW": [
    30,
    1667411467356,
    1667411876434
  ],
  "AEHH65WR5E3L6": [
    32,
    1667411673797
  ],
  "ATA61WNUAP91U": [
    33,
    1667411683834
  ],
  "A149YZJBFRDWBJ": [
    34,
    1667411770370
  ],
  "A1SL80AWG592HX": [
    35,
    1667411857667
  ],
  "A1I1IMZ2ROJY3I": [
    36,
    1667412158260
  ],
  "A1B2U0G48Y1U52": [
    37,
    1667412168250,
    1667412628125,
    1667413623675,
    1667413924624,
    1667414164762
  ],
  "A3NME80IO3UFFO": [
    38,
    1667412504694
  ],
  "A1TH9QF8FD0SF5": [
    39,
    1667412423095,
    1667412467798,
    1667412725256
  ],
  "A3RDT5DH21PVAR": [
    40,
    1667412557169
  ],
  "A1F9KLZGHE9DTA": [
    41,
    1667412665463
  ],
  "A2PTABUVOUYAH5": [
    42,
    1667413036111
  ],
  "ALAEBEEHK525U": [
    43,
    1667413197979,
    1667413278101
  ],
  "A201VG3B3F1Q40": [
    44,
    1667413385261,
    1667413892002
  ],
  "A3USIO03UXTDUT": [
    45,
    1667413691795
  ],
  "AVP8OEUW3NB4D": [
    46,
    1667413739582
  ],
  "A2ASRB2MTHDHPD": [
    48,
    1667413818790
  ],
  "A3OU2PDA0ZTT4P": [
    49,
    1667414057681,
    1667414109456
  ],
  "ATRB1HXZI3J2P": [
    50,
    1667414114770,
    1667414870410
  ],
  "A1HUQ7QA5QWM5Q": [
    999,
    999,
    999
  ],
  "A2A66W3JTSP642": [
    52,
    1667414300849
  ],
  "A98E8M4QLI9RS": [
    53,
    1667414307130
  ],
  "A2F5SKVBQQXFX0": [
    999,
    999,
    999
  ],
  "A3FJQY40AAYLDF": [
    51,
    1667414420527
  ],
  "A12K1ADYMRSWMJ": [
    54,
    1667428873368
  ],
  "A31FDAPJJ2EBGA": [
    55,
    1667414651812
  ],
  "A1W9HGZ8IKQMTW": [
    56,
    1667414589575
  ],
  "A3JJXDML3XNSQP": [
    57,
    1667414639083
  ],
  "A1OOCYEFLAJD98": [
    58,
    1667414710731
  ],
  "A5AE8MWFQVBX62": [
    58,
    1667414713167
  ],
  "A129Y082RKJN6V": [
    60,
    1667414976124,
    1667415066714,
    1667415232165
  ],
  "ASQJWS2HM0ZW9": [
    62,
    1667415007137
  ],
  "A28LKM0L8RJGB4": [
    63,
    1667415053444,
    1667415349173
  ],
  "AZ47B4BD9EFO3": [
    64,
    1667415071117,
    1667415396740,
    1667415505531,
    1667415689212
  ],
  "A3G27KBBV5G9S0": [
    65,
    1667415106844,
    1667415556856,
    1667415639867
  ],
  "A122R62XGKRFDF": [
    67,
    1667415311528,
    1667415830370
  ],
  "A2V6E7SGEU0EFD": [
    66,
    1667415204788,
    1667417087820
  ],
  "AJUM7ESRIOHVT": [
    68,
    1667416281084,
    1667416524444,
    1667416571724
  ],
  "A1NNRAT70351XO": [
    69,
    1667416289665,
    1667416510813
  ],
  "A5JWBZ2885D1N": [
    999,
    999,
    999
  ],
  "A14CZ7WXO9TOSX": [
    999,
    999,
    999
  ],
  "A1V2H0UF94ATWY": [
    70,
    1667416586668,
    1667416708536,
    1667416743675
  ],
  "AEOLA4FY5IDHE": [
    999,
    999,
    999
  ],
  "A5NE8TWS8ZV7B": [
    71,
    1667417671009
  ],
  "A23KAJRDVCVGOE": [
    72,
    1667417758197
  ],
  "A5NE8TWS8ZV7": [
    73,
    1667417764333
  ],
  "A2CFG15CY2363O": [
    74,
    1667417795015,
    1667418050919,
    1667420387607
  ],
  "A1Z4MACUJQEART": [
    75,
    1667417889389,
    1667418234709,
    1667418466623,
    1667418924996,
    1667419628154
  ],
  "A1FCTM2O25QKL2": [
    76,
    1667418154519,
    1667418618109
  ],
  "A28DTM5Q263WKQ": [
    77,
    1667418305725
  ],
  "A2Y8LZS5C81O25": [
    999,
    999,
    999
  ],
  "A26UWOGD1C89EO": [
    78,
    1667418721350,
    1667418983725,
    1667419063390
  ],
  "A1D1C4GW2Y88LW": [
    79,
    1667418730728
  ],
  "APE1BB44GIAJZ": [
    80,
    1667433527308,
    1667433900890,
    1667434170506
  ],
  "AQPFTUIJ0PW2N": [
    81,
    1667434053321,
    1667434195391
  ],
  "AQC0KPAOX4ZPL": [
    999,
    999,
    999
  ],
  "A3NAONPCTVT6P1": [
    999,
    999,
    999
  ],
  "A2QY653X9531JP": [
    83,
    1667419982737
  ],
  "AY1Q2QGK94M0Y": [
    82,
    1667420147587
  ],
  "A1LXJE5GI5JTMB": [
    84,
    1667420532464,
    1667420719965
  ],
  "A2PF6UAA5SUVD0": [
    999,
    999,
    999
  ],
  "A2M5S1RXUCVE7L": [
    85,
    1667421483479,
    1667421797919,
    1667421966357,
    1667422211321
  ],
  "ABDZ1ENQSBJ3J": [
    86,
    1667422004742
  ],
  "A3V5T6PEHRNART": [
    87,
    1667422015036,
    1667422114120
  ],
  "A2RC9V49MHS973": [
    88,
    1667422657481,
    1667423301428
  ],
  "A319QISFZHTOGY": [
    1000
  ],
  "A2R5NQLPF3GEBH": [
    89,
    1667422935622,
    1667423297457,
    1667423499485
  ],
  "A100FVY6N5ROG6": [
    1000
  ],
  "A2H3E8G9JVB3I9": [
    90,
    1667423633837,
    1667424258328,
    1667424449241,
    1667424658504
  ],
  "A18SJJAWGXVXP5": [
    91,
    1667423594746
  ],
  "A32I8CG628QXHE": [
    92,
    1667423935884,
    1667424184697
  ],
  "A290Z6QAL17PQE": [
    999,
    999,
    999
  ],
  "A2II8Y81V90TF3": [
    93,
    1667424978154,
    1667425367596
  ],
  "A3T14CFDA1ONP3": [
    94,
    1667425021244,
    1667426755337
  ],
  "A2MQPQ30Z0ETL4": [
    999,
    999,
    999
  ],
  "A2118Y81V90TF3": [
    96,
    1667425633416,
    1667425899121,
    1667425944368,
    1667425994858
  ],
  "A2BUHMLNE3LUU0": [
    95,
    1667425773436,
    1667425900929,
    1667426336350,
    1667427580185
  ],
  "AG6UL22QLCKOG": [
    999,
    999,
    999
  ],
  "A3M2BJA3PCJNE6": [
    98,
    1667426092914,
    1667427322197
  ],
  "AL9SYQEUTNJXT": [
    97,
    1667426870423
  ],
  "A1TLPOT809ERIH": [
    99,
    1667427170419,
    1667427322693,
    1667427390930,
    1667427420636,
    1667427591013,
    1667427732789,
    1667428411910
  ],
  "AXQ48BT2Y7JU9": [
    100,
    1667427232675,
    1667427922094
  ],
  "A29DB0P3TCTY3I": [
    999,
    999,
    999
  ],
  "A1P98TZA60M939": [
    101,
    1667427332610,
    1667427697832
  ],
  "A3VOV38AURK7OQ": [
    102,
    1667429010741
  ],
  "A2QRJKXTK4JDD8": [
    103,
    1667429208039
  ],
  "A2RDRBB3EHQQN8": [
    104,
    1667429273985,
    1667429712966
  ],
  "A18W98HN5XUS8H": [
    105,
    1667429129502
  ],
  "AHOLDV48HGR35": [
    106,
    1667429506824,
    1667429846973,
    1667430859590
  ],
  "A1Y0ZBE9UBJV2S": [
    999,
    999,
    999
  ],
  "A291KPOUT5WMAC": [
    107,
    1667429678658
  ],
  "A1T5FB9SJMCDG1": [
    108,
    1667430051862,
    1667430245345
  ],
  "A1RQ8PX7A0CH3C": [
    109,
    1667430687627
  ],
  "AK3GAHRZ6M6WM": [
    110,
    1667446131946
  ],
  "A2DK8IOANCNDR3": [
    111,
    1667432468810
  ],
  "A2SM1AVUVGBTH3": [
    112,
    1667432915340,
    1667433263070,
    1667433397838
  ],
  "A15PS12R80CWTU": [
    113,
    1667433514301
  ],
  "AE681FG4LT6DV": [
    114,
    1667434599791,
    1667435049620,
    1667435161112
  ],
  "A1Z6RO9XS6SQDD": [
    115,
    1667435018669
  ],
  "A13O9VR0VSV7EA": [
    116,
    1667435852207
  ],
  "A11EZAV2HRYKD5": [
    117,
    1667435975174,
    1667436218336
  ],
  "A3LHGJS3FF6QR4": [
    118,
    1667436077363,
    1667437544067
  ],
  "A2ZDWWWAZUYHFT": [
    119,
    1667436752521,
    1667437187275
  ],
  "ABVYNKCG6HU4R": [
    120,
    1667437439374,
    1667437614363
  ],
  "A3Q228ENXTJ38F": [
    999,
    999,
    999
  ],
  "AYY0UC6KCN9RW": [
    121,
    1667437718045,
    1667438536557
  ],
  "A1NQVG69U3TRDK": [
    999,
    999,
    999
  ],
  "A39FKPJOTST6HH": [
    122,
    1667438925757
  ],
  "A1Y07DW8W2UXSD": [
    123,
    1667457058212
  ],
  "AR390VVG2FJKJ": [
    124,
    999
  ],
  "AF318MABD1U0S": [
    125,
    1667440065699
  ],
  "4AI5ORKEIER5CL4": [
    126,
    1667440957835
  ],
  "A2UGQ85W3M5YU2": [
    127,
    1667441165366
  ],
  "A300G3PGI0NAO9": [
    128,
    1667441655760,
    1667442029534,
    1667444805075
  ],
  "A1DHMR97IK2SFM": [
    129,
    1667443207763
  ],
  "A2UTNTE8E0ROOU": [
    130,
    1667443517364,
    1667443707915,
    1667443836078
  ],
  "A39IAY6VBVR8FD": [
    999,
    999,
    999
  ],
  "A28E074YCKFHW0": [
    131,
    1667443892622,
    1667444320634
  ],
  "A3JRY3AL756S3P": [
    999,
    999,
    999
  ],
  "AP9YNGPNQSX7I": [
    999,
    999,
    999
  ],
  "A1O57XVTTB1PNZ": [
    133,
    1667444717789,
    1667445017368
  ],
  "A34LBDDNI3INFN": [
    132,
    1667444929427,
    1667444994694,
    1667445067184,
    1667445346632
  ],
  "A29B8DVNV2WD1I": [
    134,
    999,
    999,
    999,
    999
  ],
  "A3N692NN9S43L9": [
    135,
    1667446201983
  ],
  "A29O6FOYRB10S2": [
    999,
    999,
    999
  ],
  "A3MHP4LZ3V88NF": [
    136,
    1667447717546,
    1667447989020,
    1667448064868,
    1667448417310
  ],
  "A2DQ7Z34VFDNVX": [
    137,
    999,
    999,
    999,
    999
  ],
  "ABICECSTT3MWF": [
    138,
    1667449235591,
    999
  ],
  "A2S5ZALY1U3NUR": [
    139,
    999,
    999,
    999,
    999,
    999,
    999
  ],
  "A31YK51P992Q4L": [
    999,
    999,
    999,
    1667452473135
  ],
  "AUGML2ZY46M47": [
    999,
    999,
    999
  ],
  "AQCZJ81IS07OK": [
    999,
    999,
    1667451864242
  ],
  "AYPP7T2JCJYQ3": [
    141,
    1667452256998,
    1667452354484,
    1667452454704,
    1667452511405,
    1667452634800,
    1667452673212
  ],
  "A2HFHW1AT6CYCV": [
    999,
    999,
    999
  ],
  "A37BYKOJ588U33": [
    140,
    1667453161437
  ],
  "A9W3MMLGGWVT1": [
    142,
    1667453396681
  ],
  "A2L50T8CKUP374": [
    143,
    1667453864820,
    1667454367516
  ]
}


In [8]:
#this section reads all the relevant files: 
if (batch_data_location / 'Batch_encoding_batch_results.csv').exists():
    encoding_df=pd.read_csv(batch_data_location / 'Batch_encoding_batch_results.csv')
else: 
    print(f'no "Batch_encoding_batch_results.csv" was found in data')

if (batch_data_location / 'Batch_testing_batch_results.csv').exists():
    testing_df=pd.read_csv(batch_data_location / 'Batch_testing_batch_results.csv')
else: 
    print(f'no "Batch_testing_batch_results.csv" was found in data (probably because only the encoding session finished)')
if (batch_data_location / 'Batch_workers.csv').exists():
    workers_df=pd.read_csv(batch_data_location / 'Batch_workers.csv')
else:
    print('No AMAZON worker list was found: please make sure to upload one and name it correctly')



no "Batch_testing_batch_results.csv" was found in data (probably because only the encoding session finished)


In [9]:
workers_df.columns

Index(['Worker ID', 'Link to Individual Worker Page',
       'Number of HITs approved or rejected - Lifetime',
       'Number of HITs approved - Lifetime', 'Your Lifetime approval rate',
       'Number of HITs approved or rejected - Last 30 days',
       'Number of HITs approved - Last 30 days',
       'Your Last 30 days approval rate',
       'Number of HITs approved or rejected - Last 7 days',
       'Number of HITs approved - Last 7 days',
       'Your Last 7 days approval rate', 'CURRENT-took part in a tapseq task',
       'UPDATE-took part in a tapseq task', 'CURRENT-participated in the past',
       'UPDATE-participated in the past',
       'CURRENT-eligible for next step memory rep',
       'UPDATE-eligible for next step memory rep',
       'CURRENT-completed memory rep', 'UPDATE-completed memory rep',
       'CURRENT-completed memory encoding 24h',
       'UPDATE-completed memory encoding 24h',
       'CURRENT-completed encoding successfully ',
       'UPDATE-completed encoding

In [10]:
def find_largest_consequtive_repetition(responses_array):
#add a stop value so the loop will use the logic also if the end of the responses is all repeating: 
    responses_array=np.append(responses_array,[99])
    longest_rep_dict=dict()
    last_input=responses_array[0]
    consequtive_rep_counter=0
    for response in responses_array[1:]: 
        if response==last_input: #if consequtive rep: 
            consequtive_rep_counter=consequtive_rep_counter+1 #add to counter
        else: #once a new entry is in the vector - store the counter rep information 
            if last_input in longest_rep_dict.keys(): #if it was allready stored in our dictionary
                if longest_rep_dict[last_input]<consequtive_rep_counter: #replace the counter only if it is higher than what stored in dict
                    longest_rep_dict[last_input]=consequtive_rep_counter
            else: #this entry is not yet in the dictionary: add it 
                longest_rep_dict[last_input]=consequtive_rep_counter
            
            consequtive_rep_counter=0
            last_input=response
    return max(longest_rep_dict.values())

In [11]:
def process_worker_results(PATH_TO_BATCH,subject_name):
    cur_sub=pd.read_csv(PATH_TO_BATCH / subject_name)
    sub_demographics=cur_sub[['workID','Age','Gender']].iloc[0]

    #extract demo related information: 

    #if participant did not click on the arrow during the demo a 'demo_encoding_response.rt' colmumn wont exists: 
    #thus we will create one and fill it with nans (so everything will be consistent with other participants)
    if not('demo_encoding_response.rt' in cur_sub.columns):
        cur_sub['demo_encoding_response.rt']=np.nan
    demo_columns=['demo_encoding_loop.thisTrialN','DemoImage','DemoCorrect','demo_encoding_response.rt','demo_encoding_response.keys']

    sub_demo_information=cur_sub[demo_columns]
    empty_inds=sub_demo_information.loc[sub_demo_information.isnull().apply(lambda x: all(x), axis=1)].index
    #drop irrelevant columns: 
    sub_demo_information=sub_demo_information.drop(index=empty_inds)


    #extract the demo test columns: 
    demo_test_columns=['demo_test_response.keys','demo_test_response.corr','demo_test_response.rt','demo_test_loop.thisTrialN','DemoImage1','DemoImage2','DemoCorrectTest']
    sub_demo_test_information=cur_sub[demo_test_columns]
    empty_inds=sub_demo_test_information.loc[sub_demo_test_information.isnull().apply(lambda x: all(x), axis=1)].index
    #drop irrelevant columns: 
    sub_demo_test_information=sub_demo_test_information.drop(index=empty_inds)

    demo_df=pd.concat([sub_demo_information.reset_index(),sub_demo_test_information.reset_index()],axis=1)

    #extract real experiment related information: 

    #if participant did not click on the arrow during the encoding a 'test_encoding_response.rt' column wont exists: 
    #thus we will create one and fill it with nans (so everything will be consistent with other participants)
    if not('test_encoding_response.rt' in cur_sub.columns):
        cur_sub['test_encoding_response.rt']=np.nan


    encoding_related_columns=['test_encoding_response.keys','test_encoding_response.corr','trials.thisTrialN','target_image','pair','layer','correct','test_encoding_response.rt','key_resp_end.keys']
    sub_encoding_information=cur_sub[encoding_related_columns]
    #encoding section ends with a key press of the space key: so seperate this phase by finding this space key row
    end_of_section_ind=np.where(sub_encoding_information['key_resp_end.keys']=='space')[0][0]
    sub_encoding_information=sub_encoding_information.iloc[0:end_of_section_ind]
    #remove all the rows that precede the real encoding phase: 
    empty_inds=sub_encoding_information.loc[sub_encoding_information.isnull().apply(lambda x: all(x), axis=1)].index
    sub_encoding_information=sub_encoding_information.drop(index=empty_inds).reset_index()

    #extract real experiment TEST related information: 
    #test_related_columns=['layer','correct','test_test_response.keys','test_test_response.corr','test_test_response.rt','trials_2.thisRepN','trials_2.thisTrialN','trials_2.thisN','trials_2.thisIndex','trials_2.ran','image1','image2']
    #sub_test_information=cur_sub[test_related_columns].iloc[end_of_section_ind+2:-1]


    subject_dictionary=dict()
    subject_dictionary['demographics']=sub_demographics
    subject_dictionary['demo_df']=demo_df
    subject_dictionary['encoding_df']=sub_encoding_information
    #subject_dictionary['test_df']=sub_test_information

    return subject_dictionary



In [12]:

#get all results with Encoding information - extract the arrow attention check accuracy and RT (RT is currently not usd as a criterion)

#this section extract the list of participants from the downloaded results files (and not via the workers or session list csvs) 
# - it will create the qualification_df (a table with information on the worker ids and encoding behavior of all participants that we have files for)
all_filenames=[file for file in PATH_TO_DATA.iterdir() if 'csv' in file.name and 'ENCODING' in file.name]
print(f'current csv files:\n{all_filenames}')

qualification_for_test_df=pd.DataFrame(columns=['workerID','arrow_acc','mean_arrow_RT'])
for subject_csv in all_filenames:
    subject_dict=process_worker_results(PATH_TO_DATA,path.Path(subject_csv))
    cur_sub_encoding=subject_dict['encoding_df']

    sname=subject_csv.name.split('_')[1] #change according to actuall format. 

    #get only attention check related info: 
    cur_sub_encoding=cur_sub_encoding[['test_encoding_response.keys','correct','test_encoding_response.rt']].copy().dropna(how = 'all')
    if len(cur_sub_encoding)==0: #empty - the participant didnt respond on the arrow at all
        arrow_acc=0
        RT=nan
    else: 
        RT=cur_sub_encoding['test_encoding_response.rt'].mean()
        arrow_acc=(cur_sub_encoding['correct']==cur_sub_encoding['test_encoding_response.keys']).mean()
    sub_df=pd.DataFrame({'workerID':sname,'arrow_acc':arrow_acc,'mean_arrow_RT':RT},index=[sname])
    qualification_for_test_df=pd.concat([qualification_for_test_df,sub_df],axis=0)

#the following part update the qualification_df with information on wether the participant id exists in the amazon workers list: 

#change participants qualifications if they exists in the workers list based on thier encoding arrow accuracy
qualification_for_test_df['in_encoding_workers_list']=nan

for curr_worker_ID in qualification_for_test_df.index:
    if curr_worker_ID in encoding_df['WorkerId'].values:
        qualification_for_test_df.loc[curr_worker_ID,'in_encoding_workers_list']=int(1)
    else:
        qualification_for_test_df.loc[curr_worker_ID,'in_encoding_workers_list']=int(0)


current csv files:
[WindowsPath('c:/Users/d_abe/Desktop/human similarity database/edens github repo/Visual-memory-task/data/pilot_24_hours/batch 3/data/ENCODING_154.27.74.110_2022-10-31_08h21.33.941.csv'), WindowsPath('c:/Users/d_abe/Desktop/human similarity database/edens github repo/Visual-memory-task/data/pilot_24_hours/batch 3/data/ENCODING_A11EXIB1MVBZFJ_2022-11-02_13h36.24.962.csv'), WindowsPath('c:/Users/d_abe/Desktop/human similarity database/edens github repo/Visual-memory-task/data/pilot_24_hours/batch 3/data/ENCODING_A11F3MA5FWH6SJ_2022-10-30_12h40.40.667.csv'), WindowsPath('c:/Users/d_abe/Desktop/human similarity database/edens github repo/Visual-memory-task/data/pilot_24_hours/batch 3/data/ENCODING_A12K1ADYMRSWMJ_2022-11-02_18h41.13.368.csv'), WindowsPath('c:/Users/d_abe/Desktop/human similarity database/edens github repo/Visual-memory-task/data/pilot_24_hours/batch 3/data/ENCODING_A149YZJBFRDWBJ_2022-11-02_13h56.10.370.csv'), WindowsPath('c:/Users/d_abe/Desktop/human simi

## the section below seperate between 4 types of participants: 
    * qualified and in the workers list
    * qualified and not in the workers list 
    * disqualified and in the worker list
    * disqualified and not in the worker list

In [13]:
qualified_df=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']>=0.6) & (qualification_for_test_df['in_encoding_workers_list']==1),:].copy()
print('qualified participants')
qualified_df

qualified participants


,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list
A11EXIB1MVBZFJ,A11EXIB1MVBZFJ,1.0,1.1386,1.0
A149YZJBFRDWBJ,A149YZJBFRDWBJ,0.6,1.867767,1.0
A1F9KLZGHE9DTA,A1F9KLZGHE9DTA,1.0,0.59792,1.0
A1JJYY622DGE5L,A1JJYY622DGE5L,1.0,0.8464,1.0
A1JJYY622DGE5L,A1JJYY622DGE5L,1.0,0.49442,1.0
A1LCUPRZ0I8S3I,A1LCUPRZ0I8S3I,1.0,1.37952,1.0
A1OOCYEFLAJD98,A1OOCYEFLAJD98,1.0,0.67562,1.0
A1U0FDPQ953KXX,A1U0FDPQ953KXX,1.0,1.3084,1.0
A2A66W3JTSP642,A2A66W3JTSP642,1.0,1.0848,1.0
A2ASRB2MTHDHPD,A2ASRB2MTHDHPD,1.0,1.6313,1.0


In [14]:
qualified_df_not_in_workers_list=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']>=0.6) & (qualification_for_test_df['in_encoding_workers_list']==0),:].copy()
print('qualified participants that are not in the workers list (and thus we cant add a qualification for them)')
qualified_df_not_in_workers_list

qualified participants that are not in the workers list (and thus we cant add a qualification for them)


,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list
A11F3MA5FWH6SJ,A11F3MA5FWH6SJ,1.0,0.74672,0.0
A15B4KZ3S04HS8,A15B4KZ3S04HS8,1.0,1.03732,0.0
A1BFIV6NMCYFCL,A1BFIV6NMCYFCL,0.6,1.755667,0.0
A1L5A88C9PPK5L,A1L5A88C9PPK5L,0.8,0.788,0.0
A1LA6CIGBNDOH9,A1LA6CIGBNDOH9,1.0,0.5483,0.0
A1NFO7WZ49RZPR,A1NFO7WZ49RZPR,1.0,0.82688,0.0
A22HIX1M4QXZBB,A22HIX1M4QXZBB,1.0,0.69246,0.0
A248QG4DPULP46,A248QG4DPULP46,1.0,0.53968,0.0
A25Y3D2Y2DR8WM,A25Y3D2Y2DR8WM,0.8,1.61875,0.0
A26F7OWQB4H0CH,A26F7OWQB4H0CH,0.6,0.16346,0.0


In [15]:
disqualified_df_in_workers_list=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']<0.6) & (qualification_for_test_df['in_encoding_workers_list']==1),:].copy()
print('disqualified participants that are in the workers list:')
disqualified_df_in_workers_list

disqualified participants that are in the workers list:


,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list
A12K1ADYMRSWMJ,A12K1ADYMRSWMJ,0.4,1.282,1.0
A1SL80AWG592HX,A1SL80AWG592HX,0.4,1.78805,1.0
A2HY7GQ07YIZTT,A2HY7GQ07YIZTT,0.2,1.184,1.0
AVP8OEUW3NB4D,AVP8OEUW3NB4D,0.4,1.8267,1.0


In [16]:
disqualified_df_not_in_workers_list=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']<0.6) & (qualification_for_test_df['in_encoding_workers_list']==0),:].copy()
print('disqualified participants that are NOT in the workers list:')
disqualified_df_not_in_workers_list

disqualified participants that are NOT in the workers list:


,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list
154.27.74.110,154.27.74.110,0.4,1.286,0.0
A1618RA11ZOD8Y,A1618RA11ZOD8Y,0.2,0.952,0.0
A21TRYCBFCGZ3Y,A21TRYCBFCGZ3Y,0.4,1.1446,0.0
A244N987STMI0V,A244N987STMI0V,0.4,0.368833,0.0
A2FQLA1NF502GW,A2FQLA1NF502GW,0.2,1.7027,0.0
A2I6HVUDBTGD1,A2I6HVUDBTGD1,0.0,NaN,0.0
A2LHZMUV3Q81X0,A2LHZMUV3Q81X0,0.4,0.60784,0.0
A2R28I9SM3G0YL,A2R28I9SM3G0YL,0.2,1.877,0.0
A3NAWKOUNA608Z,A3NAWKOUNA608Z,0.2,0.5776,0.0
A3VT7TOG781BQK,A3VT7TOG781BQK,0.0,0.06196,0.0


In [17]:
qualified_df['type']='qualified in workers list'
qualified_df_not_in_workers_list['type']='qualified not in workers list'
disqualified_df_in_workers_list['type']='disqualified in workers list'
disqualified_df_not_in_workers_list['type']='disqualified not in workers list'
info_from_all_downloaded_datafiles=pd.concat([qualified_df,qualified_df_not_in_workers_list,disqualified_df_in_workers_list,disqualified_df_not_in_workers_list],axis=0)
info_summary_df=info_from_all_downloaded_datafiles.groupby('type').aggregate({'arrow_acc':'count'})
info_summary_df['worker_ids']=nan
info_summary_df['worker_ids']=info_summary_df['worker_ids'].astype(object)
if 'qualified in workers list' in info_summary_df.index: 
    info_summary_df.at['qualified in workers list','worker_ids']=qualified_df.index.to_list()
if 'qualified not in workers list' in info_summary_df.index:    
    info_summary_df.at['qualified not in workers list','worker_ids']=qualified_df_not_in_workers_list.index.to_list()
if 'disqualified in workers list' in info_summary_df.index: 
    info_summary_df.at['disqualified in workers list','worker_ids']=disqualified_df_in_workers_list.index.to_list()
if 'disqualified not in workers list' in info_summary_df.index: 
    info_summary_df.at['disqualified not in workers list','worker_ids']=disqualified_df_not_in_workers_list.index.to_list()

info_summary_df

,arrow_acc,worker_ids
type,,
disqualified in workers list,4,"[A12K1ADYMRSWMJ, A1SL80AWG592HX, A2HY7GQ07YIZT..."
disqualified not in workers list,12,"[154.27.74.110, A1618RA11ZOD8Y, A21TRYCBFCGZ3Y..."
qualified in workers list,27,"[A11EXIB1MVBZFJ, A149YZJBFRDWBJ, A1F9KLZGHE9DT..."
qualified not in workers list,37,"[A11F3MA5FWH6SJ, A15B4KZ3S04HS8, A1BFIV6NMCYFC..."


In [19]:
#update the workers list with the relevant qualification: 
new_shelf_dict=shelf_dict.copy()

#add the qualification for all eligible participants for the test session: 
ind_in_workers_df=[]
print('\nupdating qualifications for participants elligable for test session\n',list(qualified_df.index))
for worker_id in qualified_df.index:
    if worker_id in workers_df['Worker ID'].values:
        workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_testin]=1
        ind_in_workers_df.append(workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_testin].index[0])

print('\nupdating qualifications for participants elligable for test session that are not in the workers list\n',list(qualified_df_not_in_workers_list.index))
for worker_id in qualified_df_not_in_workers_list.index: 
    if worker_id in shelf_dict.keys():
        prev_entriees=shelf_dict[worker_id]
        cur_entriees=prev_entriees
        cur_entriees[0]=999
        new_shelf_dict[worker_id]=cur_entriees
        print(f'changed subject {worker_id} previous shelf entriee: {prev_entriees} to {cur_entriees} in the shelf')


#remove from experiment all those that disqualified (so they wont be able to take the test session): 
print('\nupdating qualifications for participants that were disqualified for test session and are in the workers list\n',list(disqualified_df_in_workers_list.index))
for worker_id in disqualified_df_in_workers_list.index: 
    if worker_id in workers_df['Worker ID'].values:
        workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_entire_experiment]=1
        ind_in_workers_df.append(workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_entire_experiment].index[0])
        
        if worker_id in shelf_dict.keys(): #change thier shelf entree (so they wont waste a valid csv number)
            prev_entriees=shelf_dict[worker_id]
            cur_entriees=prev_entriees.copy()
            cur_entriees[0]=999
            new_shelf_dict[worker_id]=cur_entriees
            print(f'changed subject {worker_id} previous shelf entriee: {prev_entriees} to {cur_entriees} in the shelf')


#handle those that disqualified but are not in the amazon worker list: (change thier shelf entries to 999)
print('\nhandling participants that exists in the shelf but not in the amazon workers list (reseting thier shelf entriees)\n',list(disqualified_df_not_in_workers_list.index))
for worker_id in disqualified_df_not_in_workers_list.index: 
    if worker_id in shelf_dict.keys():
        prev_entriees=shelf_dict[worker_id]
        cur_entriees=prev_entriees.copy()
        cur_entriees[0]=999
        new_shelf_dict[worker_id]=cur_entriees
        print(f'changed subject {worker_id} previous shelf entriee: {prev_entriees} to {cur_entriees} in the shelf')


#print the two relevant columns of the updated workers_list: 
print('these are all the participants that will be updated in the workers list:')
workers_df.loc[ind_in_workers_df,['Worker ID',qualification_name_for_testin,qualification_name_for_entire_experiment]]



updating qualifications for participants elligable for test session
 ['A11EXIB1MVBZFJ', 'A149YZJBFRDWBJ', 'A1F9KLZGHE9DTA', 'A1JJYY622DGE5L', 'A1JJYY622DGE5L', 'A1LCUPRZ0I8S3I', 'A1OOCYEFLAJD98', 'A1U0FDPQ953KXX', 'A2A66W3JTSP642', 'A2ASRB2MTHDHPD', 'A2M183CETUMR96', 'A31FDAPJJ2EBGA', 'A3EBIC3PKUSKCL', 'A3FJQY40AAYLDF', 'A3JJXDML3XNSQP', 'A3RDT5DH21PVAR', 'A3UZ5V4Y0K3YPV', 'A3UZ5V4Y0K3YPV', 'A3VHDQR8A9JJ4F', 'A4ANRSA55IW5Q', 'A98E8M4QLI9RS', 'AB8XECKH1JO8P', 'AE33JO53WTHZQ', 'AK4WAT44YKU7J', 'AMEBLCWTZKLS2', 'ATA61WNUAP91U', 'AVZRZOK0F26P6']

updating qualifications for participants elligable for test session that are not in the workers list
 ['A11F3MA5FWH6SJ', 'A15B4KZ3S04HS8', 'A1BFIV6NMCYFCL', 'A1L5A88C9PPK5L', 'A1LA6CIGBNDOH9', 'A1NFO7WZ49RZPR', 'A22HIX1M4QXZBB', 'A248QG4DPULP46', 'A25Y3D2Y2DR8WM', 'A26F7OWQB4H0CH', 'A2ASR7XQA7KERU', 'A2BUHMLNE3LUU0', 'A2CTBAHECDQSIX', 'A2D05J2CGBANHB', 'A2DM8RFS86D7ZK', 'A2J1DNVMJ56JG1', 'A2J57IBR2XIWLS', 'A2QCULU8SNUD1R', 'A2RS15DB7NDYIF', 'A2VY

,Worker ID,UPDATE-eligible for next step memory rep,UPDATE-completed memory rep
22,A11EXIB1MVBZFJ,1.0,NaN
62,A149YZJBFRDWBJ,1.0,NaN
225,A1F9KLZGHE9DTA,1.0,NaN
285,A1JJYY622DGE5L,1.0,NaN
285,A1JJYY622DGE5L,1.0,NaN
304,A1LCUPRZ0I8S3I,1.0,NaN
362,A1OOCYEFLAJD98,1.0,NaN
457,A1U0FDPQ953KXX,1.0,NaN
700,A2A66W3JTSP642,1.0,NaN
710,A2ASRB2MTHDHPD,1.0,NaN


In [20]:
#save the worker list to upload (to update the qualifications) -> this is what you upload to amazon to update the qualifications of relevant participants
workers_df.to_csv(batch_data_location/'Batch_workers_for_upload_after_encoding.csv',index=False)

In [21]:
workers_df.columns

Index(['Worker ID', 'Link to Individual Worker Page',
       'Number of HITs approved or rejected - Lifetime',
       'Number of HITs approved - Lifetime', 'Your Lifetime approval rate',
       'Number of HITs approved or rejected - Last 30 days',
       'Number of HITs approved - Last 30 days',
       'Your Last 30 days approval rate',
       'Number of HITs approved or rejected - Last 7 days',
       'Number of HITs approved - Last 7 days',
       'Your Last 7 days approval rate', 'CURRENT-took part in a tapseq task',
       'UPDATE-took part in a tapseq task', 'CURRENT-participated in the past',
       'UPDATE-participated in the past',
       'CURRENT-eligible for next step memory rep',
       'UPDATE-eligible for next step memory rep',
       'CURRENT-completed memory rep', 'UPDATE-completed memory rep',
       'CURRENT-completed memory encoding 24h',
       'UPDATE-completed memory encoding 24h',
       'CURRENT-completed encoding successfully ',
       'UPDATE-completed encoding

In [22]:
#this section remove participants that were assigned with an unavailble csv from the shelf (also not changing qualifications for them at all so they could come back to another encoding session in the future) 

max_csv_number=62
participants_exceeding_available_csvs=[]
for key in shelf_dict.keys():
    cur_values=shelf_dict[key]
    if (cur_values[0]>max_csv_number) and (not(cur_values[0]>=998)):
        participants_exceeding_available_csvs.append(key)
        if key in new_shelf_dict.keys():
            del new_shelf_dict[key]

print(f'the following participants were removed from the shelf dictionary due to exceeding the maximum available encoding csv file of {max_csv_number}:\nthis probably means that they did not even see the encoding of the experiment and should be considered as never participated in our\n{participants_exceeding_available_csvs}')

the following participants were removed from the shelf dictionary due to exceeding the maximum available encoding csv file of 62:
this probably means that they did not even see the encoding of the experiment and should be considered as never participated in our
['A28LKM0L8RJGB4', 'AZ47B4BD9EFO3', 'A3G27KBBV5G9S0', 'A122R62XGKRFDF', 'A2V6E7SGEU0EFD', 'AJUM7ESRIOHVT', 'A1NNRAT70351XO', 'A1V2H0UF94ATWY', 'A5NE8TWS8ZV7B', 'A23KAJRDVCVGOE', 'A5NE8TWS8ZV7', 'A2CFG15CY2363O', 'A1Z4MACUJQEART', 'A1FCTM2O25QKL2', 'A28DTM5Q263WKQ', 'A26UWOGD1C89EO', 'A1D1C4GW2Y88LW', 'APE1BB44GIAJZ', 'AQPFTUIJ0PW2N', 'A2QY653X9531JP', 'AY1Q2QGK94M0Y', 'A1LXJE5GI5JTMB', 'A2M5S1RXUCVE7L', 'ABDZ1ENQSBJ3J', 'A3V5T6PEHRNART', 'A2RC9V49MHS973', 'A2R5NQLPF3GEBH', 'A2H3E8G9JVB3I9', 'A18SJJAWGXVXP5', 'A32I8CG628QXHE', 'A2II8Y81V90TF3', 'A3T14CFDA1ONP3', 'A2118Y81V90TF3', 'A3M2BJA3PCJNE6', 'AL9SYQEUTNJXT', 'A1TLPOT809ERIH', 'AXQ48BT2Y7JU9', 'A1P98TZA60M939', 'A3VOV38AURK7OQ', 'A2QRJKXTK4JDD8', 'A2RDRBB3EHQQN8', 'A18W98HN5

In [23]:
#this section gives 999 to participants that are in the shelf but for some reason are not in the batch_results_file (so they probably saw the experiment, but we dont know how they performed during encoding so we discard them)
for worker_id in encoding_df['WorkerId']:
    if worker_id not in qualification_for_test_df.index:
        change_msg='- is not in the shelf'
        if worker_id in new_shelf_dict.keys():
            cur_values=new_shelf_dict[worker_id]
            new_values=cur_values.copy()
            new_values[0]=999
            new_shelf_dict[worker_id]=new_values
            change_msg=f'- shelf entry changed from: {cur_values} to {new_values}'

        print(f'{worker_id} did not have an encoding file ' + change_msg)

AUGML2ZY46M47 did not have an encoding file - shelf entry changed from: [999, 999, 999] to [999, 999, 999]
AL113DDABUJQ0 did not have an encoding file - shelf entry changed from: [999, 999, 999] to [999, 999, 999]
A1W9HGZ8IKQMTW did not have an encoding file - shelf entry changed from: [56, 1667414589575] to [999, 1667414589575]
A1LXJE5GI5JTMB did not have an encoding file - is not in the shelf
A3VOV38AURK7OQ did not have an encoding file - is not in the shelf
A1MT4FMXL4WRAF did not have an encoding file - shelf entry changed from: [999, 999, 999] to [999, 999, 999]
A1NQVG69U3TRDK did not have an encoding file - shelf entry changed from: [999, 999, 999] to [999, 999, 999]
A3N692NN9S43L9 did not have an encoding file - is not in the shelf
A1I1IMZ2ROJY3I did not have an encoding file - shelf entry changed from: [36, 1667412158260] to [999, 1667412158260]
A5AE8MWFQVBX6 did not have an encoding file - is not in the shelf
A2UTNTE8E0ROOU did not have an encoding file - is not in the shelf
A1

In [24]:
#this section find participants that have a valid shelf entree but are not in the worker list (at all) 
#going over the worker list and not the results list because it is possible these participant were entered into the shelf due to a previous batchwoekre
for worker_id in new_shelf_dict.keys():
    cur_values=new_shelf_dict[worker_id]
    if cur_values[0]<=max_csv_number: #if its a participant with a valid csv number
        if worker_id not in workers_df['Worker ID'].values:
            new_values=cur_values.copy()
            new_values[0]=999
            change_msg=f' - shelf entry changed from: {cur_values} to {new_values}'
            new_shelf_dict[worker_id]=new_values
            print(f'{worker_id} absent from WORKERS LIST ' + change_msg)


A2KDZLWD8RAHDM absent from WORKERS LIST  - shelf entry changed from: [3, 1667410512595] to [999, 1667410512595]
ASVSPBUO4YRBT absent from WORKERS LIST  - shelf entry changed from: [11, 1667410627238, 1667410745681, 1667410971272, 1667414162527] to [999, 1667410627238, 1667410745681, 1667410971272, 1667414162527]
A3EWBTHYFI4I6Q absent from WORKERS LIST  - shelf entry changed from: [10, 1667410796338, 1667411217538, 1667412334055] to [999, 1667410796338, 1667411217538, 1667412334055]
A23OK9EV00HCR4 absent from WORKERS LIST  - shelf entry changed from: [13, 1667410896026, 1667411400167, 1667411936244] to [999, 1667410896026, 1667411400167, 1667411936244]
A33PJ8605347GY absent from WORKERS LIST  - shelf entry changed from: [17, 1667411126127, 1667411219518] to [999, 1667411126127, 1667411219518]
A1W0K28IND6NR4 absent from WORKERS LIST  - shelf entry changed from: [20, 1667411170189] to [999, 1667411170189]
A2U50SMRZT60ZF absent from WORKERS LIST  - shelf entry changed from: [21, 1667411316

In [25]:
#this section extracts the names of the remaining valid participants in the shelf: 
valid_participants_in_shelf_for_test=[]
valid_participants_csvnumber_in_shelf_for_test=[]
for worker_id in new_shelf_dict.keys():
    cur_values=new_shelf_dict[worker_id]
    if cur_values[0]<=max_csv_number: #if its a participant with a valid csv number
        if worker_id in workers_df['Worker ID'].values:
            valid_participants_in_shelf_for_test.append(worker_id)
            valid_participants_csvnumber_in_shelf_for_test.append(cur_values[0])

valid_shelf_participants_info_df=pd.DataFrame({'Worker ID':valid_participants_in_shelf_for_test,'numeric csv':valid_participants_csvnumber_in_shelf_for_test},index=valid_participants_in_shelf_for_test)
valid_shelf_participants_info_df

,Worker ID,numeric csv
A1JJYY622DGE5L,A1JJYY622DGE5L,0
AE33JO53WTHZQ,AE33JO53WTHZQ,4
A4ANRSA55IW5Q,A4ANRSA55IW5Q,5
A11EXIB1MVBZFJ,A11EXIB1MVBZFJ,6
AMEBLCWTZKLS2,AMEBLCWTZKLS2,8
A3EBIC3PKUSKCL,A3EBIC3PKUSKCL,9
AVZRZOK0F26P6,AVZRZOK0F26P6,12
AK4WAT44YKU7J,AK4WAT44YKU7J,14
A3VHDQR8A9JJ4F,A3VHDQR8A9JJ4F,15
AB8XECKH1JO8P,AB8XECKH1JO8P,18


In [26]:
valid_shelf_participants_info_df.join(qualification_for_test_df)

,Worker ID,numeric csv,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list
A11EXIB1MVBZFJ,A11EXIB1MVBZFJ,6,A11EXIB1MVBZFJ,1.0,1.1386,1.0
A149YZJBFRDWBJ,A149YZJBFRDWBJ,34,A149YZJBFRDWBJ,0.6,1.867767,1.0
A1F9KLZGHE9DTA,A1F9KLZGHE9DTA,41,A1F9KLZGHE9DTA,1.0,0.59792,1.0
A1JJYY622DGE5L,A1JJYY622DGE5L,0,A1JJYY622DGE5L,1.0,0.8464,1.0
A1JJYY622DGE5L,A1JJYY622DGE5L,0,A1JJYY622DGE5L,1.0,0.49442,1.0
A1LCUPRZ0I8S3I,A1LCUPRZ0I8S3I,27,A1LCUPRZ0I8S3I,1.0,1.37952,1.0
A1OOCYEFLAJD98,A1OOCYEFLAJD98,58,A1OOCYEFLAJD98,1.0,0.67562,1.0
A1U0FDPQ953KXX,A1U0FDPQ953KXX,23,A1U0FDPQ953KXX,1.0,1.3084,1.0
A2A66W3JTSP642,A2A66W3JTSP642,52,A2A66W3JTSP642,1.0,1.0848,1.0
A2ASRB2MTHDHPD,A2ASRB2MTHDHPD,48,A2ASRB2MTHDHPD,1.0,1.6313,1.0


In [27]:
#this is the updated shelf: copy from here and place in the shelf location in pavlovia.org
import json
json.dumps(new_shelf_dict)

'{"A15B4KZ3S04HS8": [999, 1667149704730, 1667223830820, 1667239242566], "A1LA6CIGBNDOH9": [999, 1667156876722, 1667237966478], "A22HIX1M4QXZBB": [999, 1666804285762, 1666882028114], "A248QG4DPULP46": [999, 1666796745860, 1666875548743], "A2J1DNVMJ56JG1": [999, 1666806879557, 1666887012627], "A2J57IBR2XIWLS": [999, 1667171400527, 1667244911656, 1667248541474], "A3MIDLO5S7FU06": [999, 1667148924846, 1667227496622], "A3U0GQGAPN2DAV": [999, 1667148683750, 1667239544108], "A5P12YJP805RG": [999, 1667147891427, 1667216156787, 1667226130459], "ASNNAP90D5R1Z": [999, 1667152406322, 1667216854306, 1667237576573], "A1JJYY622DGE5L": [0, 1667410475964], "A2KDZLWD8RAHDM": [999, 1667410512595], "AE33JO53WTHZQ": [4, 1667410525427], "A4ANRSA55IW5Q": [5, 1667410579171], "A11EXIB1MVBZFJ": [6, 1667410584962], "AMEBLCWTZKLS2": [8, 1667410598167], "A3EBIC3PKUSKCL": [9, 1667410616477], "A1PN0EFQD8OHSU": [999, 999, 999], "ASVSPBUO4YRBT": [999, 1667410627238, 1667410745681, 1667410971272, 1667414162527], "A10AK

In [46]:
clean_shelf_after_test=False #change to true if you run this code after completing both encoding and test sessions and want to also resert participants that didnt come back at all... 
if clean_shelf_after_test: 


    allowed_interval_in_hours=24
    allowed_jitter=3
    allowed_interval_in_ms=[allowed_interval_in_hours-allowed_jitter,allowed_interval_in_hours+allowed_jitter]*3600*1000


    new_shelf_dict=shelf_dict.copy()
    changed_keys_list=[]
    for key in new_shelf_dict.keys():
        cur_entries=new_shelf_dict[key]
        if len(cur_entries)==2: 
            cur_entries[0]=999
            changed_keys_list.append(key)

        if len(cur_entries)>2:
            encoding_time=cur_entries[1]
            last_entree=cur_entries[-1]
            if (last_entree - encoding_time) < allowed_interval_in_ms[0]:
                cur_entries[0]=999
                changed_keys_list.append(key)


        new_shelf_dict[key]=cur_entries

    print('changed entrees for:',changed_keys_list)

    ##### print the updated shelf dictionary so you can copy it from the cell output and paste in the shelf:  (change the shelf only if you run this code after both encoding and TEST has ended) ####
    new_shelf_dict
